# Data Cleaning Listado de Listado de Beneficiarios 2022

## Autores
- José Luis Delgado Dávara
- Arturo Ortiz Aguilar
- Beltrán Valle Gutiérrez-Cortines

## Importante leer para entender

En este Notebook se trabaja con 3 listados importantes:

1. beneficiarios_22 -> Dataset con el listado de TODOS los beneficiarios de 2022.
2. Estados_Beneficiarios_2019_2022 -> Dataset sólo con los estados *únicos* encontrados en el dataset anterior.
3. Diccionario -> Emparejamiento entre ambos listados de beneficiarios
4. Dataset_Inegi -> Catálogo obtenido de Inegi

In [1]:
import pandas as pd
import seaborn as sns
from thefuzz import fuzz
from thefuzz import process
import matplotlib.pyplot as plt
import numpy as np
import os
import glob
import re
import unidecode

In [2]:
def clean_text(text):
    """
    De esta manera tenemos el texto sin espacios blancos extra y sobre todo con todas las palabras con capitalización correcta.
    """
    if pd.isna(text):
        return text
    text = text.strip()  # Eliminate white spaces
    text = text.lower()  # Convert to lowercase
    text = unidecode.unidecode(text)  # Remove accents
    text = re.sub('-.*-', '', text) #Remove what is in between - -
    text = re.sub('\s+', ' ', text)  # Eliminate extra white spaces
    text = re.sub('^\s+|\s+?$', '', text)  # Eliminate spaces at the beginning and end
    return text

# 1. Lectura y limpieza de datos

### 1.1 Beneficiarios 2022

Obtenemos el listado único de localidades y listado único de municipios con sus claves.

In [3]:
beneficiarios_22 = pd.read_csv('../../data/productores_beneficiarios 2019-2022/listado_beneficiarios_fertilizantes_2022.csv', encoding='utf-8', skiprows=0)

In [4]:
beneficiarios_22 = beneficiarios_22[~((beneficiarios_22['ENTIDAD'] == 'NACIONAL') & (beneficiarios_22['MUNICIPIO'] == 'NACIONAL') & (beneficiarios_22['LOCALIDAD'] == 'NACIONAL'))]

In [5]:
beneficiarios_22.shape

(826385, 16)

In [6]:
beneficiarios_22['ENTIDAD'].unique()

array(['GUERRERO', 'OAXACA', 'PUEBLA', 'MORELOS', 'CHIAPAS', 'DURANGO',
       'TLAXCALA', 'NAYARIT', 'ZACATECAS', nan], dtype=object)

In [7]:
beneficiarios_22.dropna(inplace=True)

In [8]:
# Obtenemos las localidades únicas en el dataset.
Localidades_22 = beneficiarios_22[['ENTIDAD', 'MUNICIPIO', 'LOCALIDAD']]
Localidades_22 = Localidades_22.drop_duplicates()


In [9]:
Localidades_22['ENTIDAD_c_benef'] = Localidades_22['ENTIDAD'].apply(clean_text)
Localidades_22['MUNICIPIO_c_benef'] = Localidades_22['MUNICIPIO'].apply(clean_text)
Localidades_22['LOCALIDAD_c_benef'] = Localidades_22['LOCALIDAD'].apply(clean_text)

In [10]:
# Creamos las dos keys de beneficiarios 2019
Localidades_22['KEY_benef_mun'] = Localidades_22['ENTIDAD_c_benef'].astype(str) + '-' + Localidades_22[
    'MUNICIPIO_c_benef'].astype(str)
Localidades_22['KEY_benef_loc'] = Localidades_22['ENTIDAD_c_benef'].astype(str) + '-' + Localidades_22[
    'MUNICIPIO_c_benef'].astype(str) + '-' + Localidades_22['LOCALIDAD_c_benef'].astype(str)

In [11]:
Localidades_22.shape

(12513, 8)

In [12]:
# Obtenemos las localidades únicas en el dataset.
Municipios_22 = beneficiarios_22[['ENTIDAD', 'MUNICIPIO']]
Municipios_22 = Municipios_22.drop_duplicates()

In [13]:
# Estandarizamos la limpieza de los datos
Municipios_22['ENTIDAD_c_benef'] = Municipios_22['ENTIDAD'].apply(clean_text)
Municipios_22['MUNICIPIO_c_benef'] = Municipios_22['MUNICIPIO'].apply(clean_text)

In [14]:
# Creamos las dos keys de beneficiarios 
Municipios_22['KEY_benef_mun'] = Municipios_22['ENTIDAD_c_benef'].astype(str) + '-' + Municipios_22[
    'MUNICIPIO_c_benef'].astype(str)

In [15]:
Municipios_22.shape

(1029, 5)

In [16]:
Municipios_22.columns

Index(['ENTIDAD', 'MUNICIPIO', 'ENTIDAD_c_benef', 'MUNICIPIO_c_benef',
       'KEY_benef_mun'],
      dtype='object')

### 1.2 INEGI 2022

Obtener listado único de municipios y listado único de localidades de inegi de 2022 con sus claves.

In [17]:
path_dataset_inegi_2022 = '../../data/inegi/dataset_inegi_clean_2022.csv'
#dataset_inegi_clean = pd.read_csv(path_dataset_inegi_2022, encoding='utf-8', dtype={'CVE_ENT': str, 'CVE_MUN': str, 'CVE_LOC': str})
dataset_inegi_2022 = pd.read_csv(path_dataset_inegi_2022)

### 1.2.1 INEGI 2022 Municipios únicos para cada año.

In [18]:
dataset_inegi_2022['KEY_inegi_municipio'] = dataset_inegi_2022['Entidad_c_inegi'].astype(str) + '-' + dataset_inegi_2022[
    'Municipio_c_inegi'].astype(str) 
dataset_inegi_2022['KEY_inegi_localidad'] = dataset_inegi_2022['Entidad_c_inegi'].astype(str) + '-' + dataset_inegi_2022[
    'Municipio_c_inegi'].astype(str) + '-' + dataset_inegi_2022['Localidad_c_inegi'].astype(str)

In [19]:
INEGI_UNIQUEMUN_2022 = dataset_inegi_2022.drop(columns=["CVE_LOC", "Localidad_inegi", "Localidad_c_inegi", "KEY_inegi_localidad"])

INEGI_UNIQUEMUN_2022 = INEGI_UNIQUEMUN_2022.drop_duplicates()

In [20]:
INEGI_UNIQUEMUN_2022.shape

(2475, 7)

In [21]:
INEGI_UNIQUEMUN_2022.columns

Index(['CVE_ENT', 'Entidad_inegi', 'CVE_MUN', 'Municipio_inegi',
       'Entidad_c_inegi', 'Municipio_c_inegi', 'KEY_inegi_municipio'],
      dtype='object')

### 1.2.1 INEGI 2022 Localidades únicas para cada año.

In [22]:
INEGI_UNIQUELOC_2022 = dataset_inegi_2022

INEGI_UNIQUELOC_2022 = INEGI_UNIQUELOC_2022.drop_duplicates()

In [23]:
INEGI_UNIQUELOC_2022.shape

(300240, 11)

# 2. Diccionario de los datasets de INEGI Y LISTADO BENEFICIARIOS 2019

El objetivo de esta sección es crear dos diccionarios de códigos según BENEFICIARIOS-MUNICIPIOS_INEGI y otro BENEFICIARIOS-LOCALIDADES_INEGI, para cada uno de los estados encontrados en inegi_2022, en este caso el número de keys es menor con lo cual no nos hará falta dividir por estado.

Para ello haremos un Left join entre Localidades_22 y el dataset de INEGI correspondiente.

In [24]:
# Crear una función para encontrar la mejor coincidencia difusa con límites entre 85 y 100 de coincidencia
def fuzzy_merge_benef2019_2022(df_benef, df_inegi, key1, key2, threshold=85, limit=1):
    """
    df_inegi: DataFrame de la izquierda (el DataFrame principal)
    df_prod: DataFrame de la derecha (el DataFrame con el que se quiere hacer el join)
    key1: Columna de la clave en df_inegi
    key2: Columna de la clave en df_prod
    threshold: Umbral de coincidencia difusa
    limit: Número de coincidencias a encontrar
    """
    s = df_inegi[key2].tolist()
    
    # Encontrar las mejores coincidencias para cada clave en df_inegi
    matches = df_benef[key1].apply(lambda x: process.extractOne(x, s, score_cutoff=threshold))


    # Crear una columna con las mejores coincidencias
    df_benef['best_match'] = [match[0] if match else None for match in matches]
    
    df_benef['match_score'] = [match[1] if match else None for match in matches]
    

    # Hacer el merge con las mejores coincidencias
    df_merged = pd.merge(df_benef, df_inegi, left_on='best_match', right_on=key2, how='left',
                         suffixes=('_benef', '_inegi'))
    
    return df_merged

In [25]:
diccionario_MUN_22 = fuzzy_merge_benef2019_2022(Municipios_22, INEGI_UNIQUEMUN_2022, 'KEY_benef_mun', 'KEY_inegi_municipio')
diccionario_MUN_22.shape

(1031, 14)

In [26]:
diccionario_MUN_22.drop(columns=['ENTIDAD', 'MUNICIPIO', 'ENTIDAD_c_benef', 'MUNICIPIO_c_benef', 'Entidad_c_inegi',
       'Municipio_c_inegi'], inplace=True)

In [27]:
diccionario_MUN_22.to_csv('../../data/productores_beneficiarios 2019-2022/diccionarios_E3/diccionario_MUN_22.csv', index=False)

In [28]:
diccionario_MUN_22_simple = pd.read_csv('../../data/productores_beneficiarios 2019-2022/diccionarios_E3/diccionario_MUN_22_simple.csv')

In [29]:
nan_rows = beneficiarios_22[beneficiarios_22.isna().any(axis=1)]

In [30]:
nan_rows

,BENEFICIARIO,ZONA,ENTIDAD,MUNICIPIO,LOCALIDAD,ESTRATIFICACIÓN,PROGRAMA,COMPONENTE,SUBCOMPONENTE,PRODUCTO,FECHA,MONTO FEDERAL,APOYO,ACTIVIDAD,ESLABÓN,CICLO AGRÍCOLA


In [31]:
# Assuming your DataFrame is named df
filtered_df = beneficiarios_22[beneficiarios_22['ENTIDAD'] == 'NACIONAL']

In [32]:
filtered_df

,BENEFICIARIO,ZONA,ENTIDAD,MUNICIPIO,LOCALIDAD,ESTRATIFICACIÓN,PROGRAMA,COMPONENTE,SUBCOMPONENTE,PRODUCTO,FECHA,MONTO FEDERAL,APOYO,ACTIVIDAD,ESLABÓN,CICLO AGRÍCOLA


### MERGE

Armamos por partes el dataset definitivo:
1. Juntamos beneficiarios_19 con el diccionario simple. (listado_beneficiario_parte_I)
2. Al df anterior juntamos las claver provenientes del catálogo de Inegi. (listado_beneficiario_parte_II)

In [33]:
beneficiarios_22.columns

Index(['BENEFICIARIO', 'ZONA', 'ENTIDAD', 'MUNICIPIO', 'LOCALIDAD',
       'ESTRATIFICACIÓN', 'PROGRAMA', 'COMPONENTE', 'SUBCOMPONENTE',
       'PRODUCTO', 'FECHA', 'MONTO FEDERAL', 'APOYO', 'ACTIVIDAD', 'ESLABÓN',
       'CICLO AGRÍCOLA'],
      dtype='object')

In [34]:
# Crear una variable KEY en listado de productores y el diccionario para hacer el join

# Clean listado beneficiarios
beneficiarios_22['ESTADO_Clean'] = beneficiarios_22['ENTIDAD'].apply(clean_text)
beneficiarios_22['MUNICIPIO_Clean'] = beneficiarios_22['MUNICIPIO'].apply(clean_text)

# Create KEY in listado beneficiarios
beneficiarios_22['Estado-mun-KEY'] = beneficiarios_22['ESTADO_Clean'].astype(str) + '-' + beneficiarios_22[
    'MUNICIPIO_Clean'].astype(str)

In [35]:
# Hacer el join de la Parte I
listado_beneficiarios_parte_I = pd.merge(beneficiarios_22, diccionario_MUN_22_simple, left_on="Estado-mun-KEY",
                                        right_on="KEY_benef_mun", how='left', suffixes=('_benef', '_inegi'))

In [36]:
listado_beneficiarios_parte_I.columns

Index(['BENEFICIARIO', 'ZONA', 'ENTIDAD', 'MUNICIPIO', 'LOCALIDAD',
       'ESTRATIFICACIÓN', 'PROGRAMA', 'COMPONENTE', 'SUBCOMPONENTE',
       'PRODUCTO', 'FECHA', 'MONTO FEDERAL', 'APOYO', 'ACTIVIDAD', 'ESLABÓN',
       'CICLO AGRÍCOLA', 'ESTADO_Clean', 'MUNICIPIO_Clean', 'Estado-mun-KEY',
       'KEY_benef_mun', 'KEY_inegi_municipio'],
      dtype='object')

In [37]:
# Hacer el join de la Parte II
listado_beneficiarios_parte_II = pd.merge(listado_beneficiarios_parte_I, INEGI_UNIQUEMUN_2022, left_on="KEY_benef_mun",
                                        right_on="KEY_inegi_municipio", how='left', suffixes=('_benef', '_inegi'))

In [38]:
listado_beneficiarios_parte_II

,BENEFICIARIO,ZONA,ENTIDAD,MUNICIPIO,LOCALIDAD,ESTRATIFICACIÓN,PROGRAMA,COMPONENTE,SUBCOMPONENTE,PRODUCTO,...,Estado-mun-KEY,KEY_benef_mun,KEY_inegi_municipio_benef,CVE_ENT,Entidad_inegi,CVE_MUN,Municipio_inegi,Entidad_c_inegi,Municipio_c_inegi,KEY_inegi_municipio_inegi
0,ABAC NERI JESUS CARLOS,CENTRO PAÍS,GUERRERO,COYUCA DE BENÍTEZ,PUEBLO VIEJO,Alto,FERTILIZANTES,FERTILIZANTES,FERTILIZANTES,Cultivo de maíz grano blanco,...,guerrero-coyuca de benitez,guerrero-coyuca de benitez,guerrero-coyuca de benitez,12.0,Guerrero,21.0,Coyuca de Benítez,guerrero,coyuca de benitez,guerrero-coyuca de benitez
1,ABAD ALARCON ELIAZAR,CENTRO PAÍS,GUERRERO,COYUCA DE BENÍTEZ,PUEBLO VIEJO,Alto,FERTILIZANTES,FERTILIZANTES,FERTILIZANTES,Cultivo de maíz grano blanco,...,guerrero-coyuca de benitez,guerrero-coyuca de benitez,guerrero-coyuca de benitez,12.0,Guerrero,21.0,Coyuca de Benítez,guerrero,coyuca de benitez,guerrero-coyuca de benitez
2,ABAD ALARCON EZEQUIEL,CENTRO PAÍS,GUERRERO,COYUCA DE BENÍTEZ,AGUA ZARCA,Alto,FERTILIZANTES,FERTILIZANTES,FERTILIZANTES,Cultivo de maíz grano blanco,...,guerrero-coyuca de benitez,guerrero-coyuca de benitez,guerrero-coyuca de benitez,12.0,Guerrero,21.0,Coyuca de Benítez,guerrero,coyuca de benitez,guerrero-coyuca de benitez
3,ABAD ALARCON FULGENCIO,CENTRO PAÍS,GUERRERO,COYUCA DE BENÍTEZ,AGUA ZARCA,Alto,FERTILIZANTES,FERTILIZANTES,FERTILIZANTES,Cultivo de maíz grano blanco,...,guerrero-coyuca de benitez,guerrero-coyuca de benitez,guerrero-coyuca de benitez,12.0,Guerrero,21.0,Coyuca de Benítez,guerrero,coyuca de benitez,guerrero-coyuca de benitez
4,ABAD ALARCON ROMALDA,CENTRO PAÍS,GUERRERO,COYUCA DE BENÍTEZ,YERBASANTITA,Alto,FERTILIZANTES,FERTILIZANTES,FERTILIZANTES,Cultivo de maíz grano blanco,...,guerrero-coyuca de benitez,guerrero-coyuca de benitez,guerrero-coyuca de benitez,12.0,Guerrero,21.0,Coyuca de Benítez,guerrero,coyuca de benitez,guerrero-coyuca de benitez
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
834066,ZURITA ZURITA ROLANDO,SUR SURESTE,OAXACA,SAN SEBASTIÁN TECOMAXTLAHUACA,SAN MATEO TUNUCHI,Alto,FERTILIZANTES,FERTILIZANTES,FERTILIZANTES,Milpa,...,oaxaca-san sebastian tecomaxtlahuaca,oaxaca-san sebastian tecomaxtlahuaca,oaxaca-san sebastian tecomaxtlahuaca,20.0,Oaxaca,348.0,San Sebastián Tecomaxtlahuaca,oaxaca,san sebastian tecomaxtlahuaca,oaxaca-san sebastian tecomaxtlahuaca
834067,ZURITA ZURITA SIMEON,SUR SURESTE,OAXACA,SAN JUAN LACHAO,SAN JUAN LACHAO,Alto,FERTILIZANTES,FERTILIZANTES,FERTILIZANTES,Cultivo de maíz grano blanco,...,oaxaca-san juan lachao,oaxaca-san juan lachao,oaxaca-san juan lachao,20.0,Oaxaca,202.0,San Juan Lachao,oaxaca,san juan lachao,oaxaca-san juan lachao
834068,ZURITA ZURITA ZENAIDA,SUR SURESTE,OAXACA,SANTA MARÍA OZOLOTEPEC,SANTA MARÍA OZOLOTEPEC,Alto,FERTILIZANTES,FERTILIZANTES,FERTILIZANTES,Milpa,...,oaxaca-santa maria ozolotepec,oaxaca-santa maria ozolotepec,oaxaca-santa maria ozolotepec,20.0,Oaxaca,424.0,Santa María Ozolotepec,oaxaca,santa maria ozolotepec,oaxaca-santa maria ozolotepec
834069,ZURROZA RUEDA APOLINAR,SUR SURESTE,OAXACA,SANTO DOMINGO CHIHUITÁN,SANTO DOMINGO CHIHUITÁN,Medio,FERTILIZANTES,FERTILIZANTES,FERTILIZANTES,Cultivo de maíz grano blanco,...,oaxaca-santo domingo chihuitan,oaxaca-santo domingo chihuitan,oaxaca-santo domingo chihuitan,20.0,Oaxaca,508.0,Santo Domingo Chihuitán,oaxaca,santo domingo chihuitan,oaxaca-santo domingo chihuitan


In [39]:
listado_beneficiarios_parte_II.columns

Index(['BENEFICIARIO', 'ZONA', 'ENTIDAD', 'MUNICIPIO', 'LOCALIDAD',
       'ESTRATIFICACIÓN', 'PROGRAMA', 'COMPONENTE', 'SUBCOMPONENTE',
       'PRODUCTO', 'FECHA', 'MONTO FEDERAL', 'APOYO', 'ACTIVIDAD', 'ESLABÓN',
       'CICLO AGRÍCOLA', 'ESTADO_Clean', 'MUNICIPIO_Clean', 'Estado-mun-KEY',
       'KEY_benef_mun', 'KEY_inegi_municipio_benef', 'CVE_ENT',
       'Entidad_inegi', 'CVE_MUN', 'Municipio_inegi', 'Entidad_c_inegi',
       'Municipio_c_inegi', 'KEY_inegi_municipio_inegi'],
      dtype='object')

In [40]:
listado_beneficiarios_parte_II = listado_beneficiarios_parte_II.drop(columns=['ESTADO_Clean', 'MUNICIPIO_Clean', 'Estado-mun-KEY', 'KEY_inegi_municipio_benef', 'Entidad_c_inegi','Municipio_c_inegi', 'KEY_inegi_municipio_inegi', 'ESTRATIFICACIÓN', 'PROGRAMA', 'COMPONENTE', 'SUBCOMPONENTE', 'APOYO', 'ACTIVIDAD', 'ESLABÓN'])

In [41]:
listado_beneficiarios_parte_II

,BENEFICIARIO,ZONA,ENTIDAD,MUNICIPIO,LOCALIDAD,PRODUCTO,FECHA,MONTO FEDERAL,CICLO AGRÍCOLA,KEY_benef_mun,CVE_ENT,Entidad_inegi,CVE_MUN,Municipio_inegi
0,ABAC NERI JESUS CARLOS,CENTRO PAÍS,GUERRERO,COYUCA DE BENÍTEZ,PUEBLO VIEJO,Cultivo de maíz grano blanco,2022-06-26,12244.18,PV2022,guerrero-coyuca de benitez,12.0,Guerrero,21.0,Coyuca de Benítez
1,ABAD ALARCON ELIAZAR,CENTRO PAÍS,GUERRERO,COYUCA DE BENÍTEZ,PUEBLO VIEJO,Cultivo de maíz grano blanco,2022-06-26,6122.09,PV2022,guerrero-coyuca de benitez,12.0,Guerrero,21.0,Coyuca de Benítez
2,ABAD ALARCON EZEQUIEL,CENTRO PAÍS,GUERRERO,COYUCA DE BENÍTEZ,AGUA ZARCA,Cultivo de maíz grano blanco,2022-07-12,12244.18,PV2022,guerrero-coyuca de benitez,12.0,Guerrero,21.0,Coyuca de Benítez
3,ABAD ALARCON FULGENCIO,CENTRO PAÍS,GUERRERO,COYUCA DE BENÍTEZ,AGUA ZARCA,Cultivo de maíz grano blanco,2022-07-12,12244.18,PV2022,guerrero-coyuca de benitez,12.0,Guerrero,21.0,Coyuca de Benítez
4,ABAD ALARCON ROMALDA,CENTRO PAÍS,GUERRERO,COYUCA DE BENÍTEZ,YERBASANTITA,Cultivo de maíz grano blanco,2022-06-20,12244.18,PV2022,guerrero-coyuca de benitez,12.0,Guerrero,21.0,Coyuca de Benítez
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
834066,ZURITA ZURITA ROLANDO,SUR SURESTE,OAXACA,SAN SEBASTIÁN TECOMAXTLAHUACA,SAN MATEO TUNUCHI,Milpa,2022-08-18,7435.60,PV2022,oaxaca-san sebastian tecomaxtlahuaca,20.0,Oaxaca,348.0,San Sebastián Tecomaxtlahuaca
834067,ZURITA ZURITA SIMEON,SUR SURESTE,OAXACA,SAN JUAN LACHAO,SAN JUAN LACHAO,Cultivo de maíz grano blanco,2022-08-03,7435.60,PV2022,oaxaca-san juan lachao,20.0,Oaxaca,202.0,San Juan Lachao
834068,ZURITA ZURITA ZENAIDA,SUR SURESTE,OAXACA,SANTA MARÍA OZOLOTEPEC,SANTA MARÍA OZOLOTEPEC,Milpa,2022-08-11,7435.60,PV2022,oaxaca-santa maria ozolotepec,20.0,Oaxaca,424.0,Santa María Ozolotepec
834069,ZURROZA RUEDA APOLINAR,SUR SURESTE,OAXACA,SANTO DOMINGO CHIHUITÁN,SANTO DOMINGO CHIHUITÁN,Cultivo de maíz grano blanco,2022-08-08,7435.60,PV2022,oaxaca-santo domingo chihuitan,20.0,Oaxaca,508.0,Santo Domingo Chihuitán


In [42]:
listado_beneficiarios_parte_II.columns

Index(['BENEFICIARIO', 'ZONA', 'ENTIDAD', 'MUNICIPIO', 'LOCALIDAD', 'PRODUCTO',
       'FECHA', 'MONTO FEDERAL', 'CICLO AGRÍCOLA', 'KEY_benef_mun', 'CVE_ENT',
       'Entidad_inegi', 'CVE_MUN', 'Municipio_inegi'],
      dtype='object')

In [43]:
listado_beneficiarios_parte_II.to_csv('../../data/listados_completos/listado_beneficiarios_2022.csv', index=False)

In [44]:
listado_beneficiarios_parte_II.shape

(834071, 14)

In [45]:
listado_beneficiarios_parte_II.isna().sum()

BENEFICIARIO          0
ZONA                  0
ENTIDAD               0
MUNICIPIO             0
LOCALIDAD             0
PRODUCTO              0
FECHA                 0
MONTO FEDERAL         0
CICLO AGRÍCOLA        0
KEY_benef_mun         0
CVE_ENT            5923
Entidad_inegi      5923
CVE_MUN            5923
Municipio_inegi    5923
dtype: int64

# 3. Join por entidad (Divide y Vencerás)

### 3.1.1 Join de localidades - PUEBLA

In [44]:
Localidades_22_PUEBLA = Localidades_22[Localidades_22["ENTIDAD"] == "PUEBLA"]

dataset_inegi_uniqueloc_PUEBLA = INEGI_UNIQUELOC_2022[INEGI_UNIQUELOC_2022['Entidad_inegi'] == 'Puebla']

In [45]:
Localidades_22_PUEBLA = fuzzy_merge_benef2019_2022(Localidades_22_PUEBLA, dataset_inegi_uniqueloc_PUEBLA, 'KEY_benef_loc', 'KEY_inegi_localidad')
Localidades_22_PUEBLA.shape

/var/folders/ng/n9ng0wy92t90_49j662nh02c0000gn/T/ipykernel_86460/734782849.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_benef['best_match'] = [match[0] if match else None for match in matches]
/var/folders/ng/n9ng0wy92t90_49j662nh02c0000gn/T/ipykernel_86460/734782849.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_benef['match_score'] = [match[1] if match else None for match in matches]


(744, 21)

### 3.2.2 Join de localidades - CHIAPAS

In [46]:
Localidades_22_CHIAPAS = Localidades_22[Localidades_22["ENTIDAD"] == "CHIAPAS"]

dataset_inegi_uniqueloc_CHIAPAS = INEGI_UNIQUELOC_2022[INEGI_UNIQUELOC_2022['Entidad_inegi'] == 'Chiapas']

In [47]:
Localidades_22_CHIAPAS = fuzzy_merge_benef2019_2022(Localidades_22_CHIAPAS, dataset_inegi_uniqueloc_CHIAPAS, 'KEY_benef_loc', 'KEY_inegi_localidad')
Localidades_22_CHIAPAS.shape

### 3.2.3 Join de localidades - DURANGO

In [ ]:
Localidades_22_DURANGO = Localidades_22[Localidades_22["ENTIDAD"] == "DURANGO"]

dataset_inegi_uniqueloc_DURANGO = INEGI_UNIQUELOC_2022[INEGI_UNIQUELOC_2022['Entidad_inegi'] == 'Durango']

In [ ]:
Localidades_22_DURANGO = fuzzy_merge_benef2019_2022(Localidades_22_DURANGO, dataset_inegi_uniqueloc_DURANGO, 'KEY_benef_loc', 'KEY_inegi_localidad')
Localidades_22_DURANGO.shape

/var/folders/ng/n9ng0wy92t90_49j662nh02c0000gn/T/ipykernel_34875/734782849.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_benef['best_match'] = [match[0] if match else None for match in matches]
/var/folders/ng/n9ng0wy92t90_49j662nh02c0000gn/T/ipykernel_34875/734782849.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_benef['match_score'] = [match[1] if match else None for match in matches]


(678, 21)

### 3.2.4 Join de localidades - MORELOS

In [ ]:
Localidades_22_MORELOS = Localidades_22[Localidades_22["ENTIDAD"] == "MORELOS"]

dataset_inegi_uniqueloc_MORELOS = INEGI_UNIQUELOC_2022[INEGI_UNIQUELOC_2022['Entidad_inegi'] == 'Morelos']

In [ ]:
Localidades_22_MORELOS = fuzzy_merge_benef2019_2022(Localidades_22_MORELOS, dataset_inegi_uniqueloc_MORELOS, 'KEY_benef_loc', 'KEY_inegi_localidad')
Localidades_22_MORELOS.shape

/var/folders/ng/n9ng0wy92t90_49j662nh02c0000gn/T/ipykernel_34875/734782849.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_benef['best_match'] = [match[0] if match else None for match in matches]
/var/folders/ng/n9ng0wy92t90_49j662nh02c0000gn/T/ipykernel_34875/734782849.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_benef['match_score'] = [match[1] if match else None for match in matches]


(271, 21)

### 3.2.5 Join de localidades - GUERRERO

In [ ]:
Localidades_22_GUERRERO = Localidades_22[Localidades_22["ENTIDAD"] == "GUERRERO"]

dataset_inegi_uniqueloc_GUERRERO = INEGI_UNIQUELOC_2022[INEGI_UNIQUELOC_2022['Entidad_inegi'] == 'Guerrero']

In [ ]:
Localidades_22_GUERRERO = fuzzy_merge_benef2019_2022(Localidades_22_GUERRERO, dataset_inegi_uniqueloc_GUERRERO, 'KEY_benef_loc', 'KEY_inegi_localidad')
Localidades_22_GUERRERO.shape

/var/folders/ng/n9ng0wy92t90_49j662nh02c0000gn/T/ipykernel_34875/734782849.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_benef['best_match'] = [match[0] if match else None for match in matches]
/var/folders/ng/n9ng0wy92t90_49j662nh02c0000gn/T/ipykernel_34875/734782849.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_benef['match_score'] = [match[1] if match else None for match in matches]


(4847, 21)

### 3.2.6 Join de localidades - NAYARIT

In [ ]:
Localidades_22_NAYARIT = Localidades_22[Localidades_22["ENTIDAD"] == "NAYARIT"]

dataset_inegi_uniqueloc_NAYARIT = INEGI_UNIQUELOC_2022[INEGI_UNIQUELOC_2022['Entidad_inegi'] == 'Nayarit']

In [ ]:
Localidades_22_NAYARIT = fuzzy_merge_benef2019_2022(Localidades_22_NAYARIT, dataset_inegi_uniqueloc_NAYARIT, 'KEY_benef_loc', 'KEY_inegi_localidad')
Localidades_22_NAYARIT.shape

/var/folders/ng/n9ng0wy92t90_49j662nh02c0000gn/T/ipykernel_34875/734782849.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_benef['best_match'] = [match[0] if match else None for match in matches]
/var/folders/ng/n9ng0wy92t90_49j662nh02c0000gn/T/ipykernel_34875/734782849.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_benef['match_score'] = [match[1] if match else None for match in matches]


(356, 21)

### 3.2.7 Join de localidades - OAXACA

In [ ]:
Localidades_22_OAXACA = Localidades_22[Localidades_22["ENTIDAD"] == "OAXACA"]

dataset_inegi_uniqueloc_OAXACA = INEGI_UNIQUELOC_2022[INEGI_UNIQUELOC_2022['Entidad_inegi'] == 'Oaxaca']

In [ ]:
Localidades_22_OAXACA = fuzzy_merge_benef2019_2022(Localidades_22_OAXACA, dataset_inegi_uniqueloc_OAXACA, 'KEY_benef_loc', 'KEY_inegi_localidad')
Localidades_22_OAXACA.shape

/var/folders/ng/n9ng0wy92t90_49j662nh02c0000gn/T/ipykernel_34875/734782849.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_benef['best_match'] = [match[0] if match else None for match in matches]
/var/folders/ng/n9ng0wy92t90_49j662nh02c0000gn/T/ipykernel_34875/734782849.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_benef['match_score'] = [match[1] if match else None for match in matches]


(1800, 21)

### 3.2.8 Join de localidades - TLAXCALA

In [ ]:
Localidades_22_TLAXCALA = Localidades_22[Localidades_22["ENTIDAD"] == "TLAXCALA"]

dataset_inegi_uniqueloc_TLAXCALA = INEGI_UNIQUELOC_2022[INEGI_UNIQUELOC_2022['Entidad_inegi'] == 'Tlaxcala']

In [ ]:
Localidades_22_TLAXCALA = fuzzy_merge_benef2019_2022(Localidades_22_TLAXCALA, dataset_inegi_uniqueloc_TLAXCALA, 'KEY_benef_loc', 'KEY_inegi_localidad')
Localidades_22_TLAXCALA.shape

/var/folders/ng/n9ng0wy92t90_49j662nh02c0000gn/T/ipykernel_34875/734782849.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_benef['best_match'] = [match[0] if match else None for match in matches]
/var/folders/ng/n9ng0wy92t90_49j662nh02c0000gn/T/ipykernel_34875/734782849.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_benef['match_score'] = [match[1] if match else None for match in matches]


(412, 21)

### 3.2.9 Join de localidades - ZACATECAS

In [ ]:
Localidades_22_ZACATECAS = Localidades_22[Localidades_22["ENTIDAD"] == "ZACATECAS"]

dataset_inegi_uniqueloc_ZACATECAS = INEGI_UNIQUELOC_2022[INEGI_UNIQUELOC_2022['Entidad_inegi'] == 'Zacatecas']

In [ ]:
Localidades_22_ZACATECAS = fuzzy_merge_benef2019_2022(Localidades_22_ZACATECAS, dataset_inegi_uniqueloc_ZACATECAS, 'KEY_benef_loc', 'KEY_inegi_localidad')
Localidades_22_ZACATECAS.shape

/var/folders/ng/n9ng0wy92t90_49j662nh02c0000gn/T/ipykernel_34875/734782849.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_benef['best_match'] = [match[0] if match else None for match in matches]
/var/folders/ng/n9ng0wy92t90_49j662nh02c0000gn/T/ipykernel_34875/734782849.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_benef['match_score'] = [match[1] if match else None for match in matches]


(590, 21)